# Crude oil basic data analysis

## Data cleanup and storage.

We will clean the data store it into a sqlite database. Usually I use Pandas to do my analysis and will do so here. Unfortunately, as we will see later, the amount of data in our database will explode and no longer fit in memroy. As a result using a relational database like Sqlite will help us do operations without running into huge performance problems.

Using Sqlite is useful beacuse it saves the data in a coherent and safe way. This also means we need to either stick with its native language, SQL, or use a on-demand library like SqlAlchemy to process the data. 

In [1]:
# Import our relevant libraries here
# Try to keep them here instead of scattered all over the notebook....
import pandas as pd  # For easy data manipulations
from datetime import datetime, date  # For date and time functionality
import sqlalchemy as sa  # For database functionality

# import plotly files. These will help with plotting in an
# interactive way. 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout, Histogram
init_notebook_mode(connected=True)

In [2]:
# Create a database file using sqlite
engine = sa.create_engine('sqlite:///cl_basic_data_analysis.db')
# Make it easier to download data by making a generatic function
def data_from_table(table_name, index_col=None):
    return pd.read_sql_query('SELECT * FROM {}'.format(table_name), 
                             con=engine, index_col=index_col)

In [3]:
# Load the data and clean it a bit
# The date is in a format that needs to be converted to a datetime object
historic_data_file = 'Historic_Crude_Oil_Data.csv'
pdf = pd.read_csv(historic_data_file, index_col=0)
pdf['Date'] = pdf['Date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
pdf.drop('FileName', axis=1, inplace=True)

In [4]:
# Look at the datatypes for this table
pdf.dtypes

Date             datetime64[ns]
Open                    float64
High                    float64
Low                     float64
Close                   float64
Volume                    int64
OpenInt                   int64
contract_name            object
dtype: object

In [5]:
# Look at some data
pdf.head()

,Date,Open,High,Low,Close,Volume,OpenInt,contract_name
0,1983-03-30,29.01,29.56,29.01,29.40,949,470,CL1983-06
1,1983-03-31,29.40,29.60,29.25,29.29,521,523,CL1983-06
2,1983-04-04,29.30,29.70,29.29,29.44,156,583,CL1983-06
3,1983-04-05,29.50,29.80,29.50,29.71,175,623,CL1983-06
4,1983-04-06,29.90,29.92,29.65,29.90,392,640,CL1983-06


In [6]:
# We can look at the data an see a few basic trends using the describe tool
# that comes with Pandas
pdf.describe()

,Open,High,Low,Close,Volume,OpenInt
count,347810.000000,347810.000000,347810.000000,347810.000000,3.478100e+05,347810.000000
mean,56.706936,56.854985,56.550690,56.712603,7.324991e+03,18622.319744
std,30.600976,30.650910,30.541844,30.603532,3.795087e+04,45760.484265
min,10.000000,10.650000,9.750000,10.420000,0.000000e+00,0.000000
25%,23.050000,23.090000,22.990000,23.040000,0.000000e+00,105.250000
50%,58.790000,59.000000,58.550000,58.780000,1.000000e+01,2929.000000
75%,84.860000,84.950000,84.760000,84.870000,9.820000e+02,15752.000000
max,148.000000,148.600000,146.860000,146.940000,1.311000e+06,642793.000000


In [7]:
def update_cl_data(pdf, table_name, index_name=None):
    """
    This function will delete an old database and create a new one
    in its place. Instead of updating, we wipe the data because we
    naiively do not know if the table has updated columns.
    BE CAREFUL! Do not run this accidently with other dataframes. 
    """

    try:
        # Connect to the database
        conn = engine.connect()  
        # Use metadata to keep track of database information
        metadata = sa.MetaData(bind=conn)  
        # Get database information using reflect function
        metadata.reflect()  
        if 'cl_data' in metadata.tables:
            # Drop table so that we don't make duplicates
            conn.execute('DROP TABLE {}'.format(table_name))  
        if index_name is not None:
            # Make sure index column is named for comparing  
            # data moved to and from database.
            # Pandas doesn't require a named index but 
            # SQL databases do. 
            pdf.index.name = 'index'  
                                      
        # Upload data to database
        pdf.to_sql('cl_data', con=conn) 
    finally:
        conn.close()
        
update_cl_data(pdf, 'cl_data', index_name='index')  # Update our table   

In [8]:
# Another way to read table
# Slightly slowly than pandas "read_sql_query" function
df_sql = pd.read_sql_table('cl_data', con=engine, index_col='index')

In [9]:
# Second way to read table
# This uses pandas "read_sql_query" function
df_query = data_from_table('cl_data', 'index')

In [10]:
# Make sure data has not been transformed in the upload and download from the database
df_query.head()

,Date,Open,High,Low,Close,Volume,OpenInt,contract_name
index,,,,,,,,
0,1983-03-30 00:00:00.000000,29.01,29.56,29.01,29.40,949,470,CL1983-06
1,1983-03-31 00:00:00.000000,29.40,29.60,29.25,29.29,521,523,CL1983-06
2,1983-04-04 00:00:00.000000,29.30,29.70,29.29,29.44,156,583,CL1983-06
3,1983-04-05 00:00:00.000000,29.50,29.80,29.50,29.71,175,623,CL1983-06
4,1983-04-06 00:00:00.000000,29.90,29.92,29.65,29.90,392,640,CL1983-06


In [11]:
# Notice Sqlite didn't save the Date as a Datetime object and
# instead is a string. Sad days
dt_example = df_query['Date'].iloc[0]
dt_example, type(dt_example)

('1983-03-30 00:00:00.000000', str)

## Adding columns for data analysis

We can split this table into more columns. This wil be useful to do grouping of our data to find trends. 

The following columns can be split into parts:
   + Date: Year, Month, Day
   + Contract name: Symbol, Year, Month

In [12]:
# Date: Year, Month, Day
f = lambda x: x.split(' ')[0].split('-')
df = pd.DataFrame(list(df_query['Date'].map(f)), 
                  columns=('year', 'month', 'day'))
df['year'] = df['year'].map(int)
df['month'] = df['month'].map(int)
df['day'] = df['day'].map(int)
pdf = pd.concat((df_query, df), axis=1).drop('Date', axis=1)
pdf.head(2)

,Open,High,Low,Close,Volume,OpenInt,contract_name,year,month,day
index,,,,,,,,,,
0,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30
1,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31


In [13]:
# Date: Year, Month, Day
f = lambda x: (x[:2], int(x[2:6]), int(x[-2:]))
df = pd.DataFrame(list(df_query['contract_name'].map(f)), 
                  columns=('contract_symbol', 
                           'contract_year', 
                           'contract_month'))
more_columns_data = pd.concat((pdf, df), axis=1)
more_columns_data.head(2)

,Open,High,Low,Close,Volume,OpenInt,contract_name,year,month,day,contract_symbol,contract_year,contract_month
index,,,,,,,,,,,,,
0,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30,CL,1983,6
1,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31,CL,1983,6


In [14]:
# For the sake of consistency lets keep all the colun names in lowercase
columns = list(map(lambda x: x.lower(), more_columns_data.columns))
more_columns_data.columns = columns
more_columns_data.head(2)

,open,high,low,close,volume,openint,contract_name,year,month,day,contract_symbol,contract_year,contract_month
index,,,,,,,,,,,,,
0,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30,CL,1983,6
1,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31,CL,1983,6


In [15]:
# Make sure we have the expected datatypes
more_columns_data.dtypes

open               float64
high               float64
low                float64
close              float64
volume               int64
openint              int64
contract_name       object
year                 int64
month                int64
day                  int64
contract_symbol     object
contract_year        int64
contract_month       int64
dtype: object

In [16]:
# Finally upload new dataset to our database
update_cl_data(more_columns_data, 'cl_data', 'index')

In [17]:
# Grab our data from the database again
df_query = data_from_table('cl_data', 'index')
df_query.head()

,open,high,low,close,volume,openint,contract_name,year,month,day,contract_symbol,contract_year,contract_month
index,,,,,,,,,,,,,
0,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30,CL,1983,6
1,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31,CL,1983,6
2,29.30,29.70,29.29,29.44,156,583,CL1983-06,1983,4,4,CL,1983,6
3,29.50,29.80,29.50,29.71,175,623,CL1983-06,1983,4,5,CL,1983,6
4,29.90,29.92,29.65,29.90,392,640,CL1983-06,1983,4,6,CL,1983,6


In [18]:
# Make sure the data tyeps are coherent
df_query.dtypes

open               float64
high               float64
low                float64
close              float64
volume               int64
openint              int64
contract_name       object
year                 int64
month                int64
day                  int64
contract_symbol     object
contract_year        int64
contract_month       int64
dtype: object